In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



A company decides to use Facebook for marketing their new product. Facebook is capable of targeting adds very precisely to the right persons if you know who those right persons are. Because the product is new the company doesn’t know who the potential buyers are. The company decides to make a test-add in Facebook to find out which type of users clicks the “purchase button” in the add, that is, what age or gender they are and what income level they might have. When a user sees a add in Facebook, Facebook can provide all this information about the user. After the test-add, the company has collected the following information about all the users who have seen the add in their Facbook feed :

    'User ID': unique identification for consumer.
    'Age': cutomer age in years
    'Estimated Salary': Avg. Income of consumer
    'Gender': female = 0 and male = 1
    ‘clicked_to_purhase’: 0 no or 1 yes

TASK 1: Find the rules for Facebook for targetting the adds more accurately: (e.g IF potential buyer is a woman AND <30 year old AND avg. income is <78000, the user has the highest propability for purchasing the product, propability=82,4. These figures were just examples.

TASK 2: Find most accurate model for predicting if a user seeing the add will click the “purchase”-button based on Age,Salary and Gender. ( Create different models with different tools and find the most accurate model. )


In [ ]:
import pandas
import sklearn
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import datatable as dt # pip install datatble
!pip install pydotplus
import pydotplus

In [ ]:
# open data
data = pandas.read_csv('../input/socialnetworkads/Social_Network_Ads.csv', header=0)

#visualize
data.head(10)

In [ ]:
data.info()

In [ ]:
data.sample(10)

In [ ]:
data['Purchased'].value_counts()

There are 143 purchases in the data vs 257 not purchased, when 1=purchase and 0=not purchased.

In [ ]:
data['Gender'].value_counts()

0 = female and 1 = male, so there are 204 men in the data and 196 women.

In [ ]:
data['Age'].value_counts()

In [ ]:
data.Age.value_counts(normalize=True).plot(kind="bar", figsize=(12,10)) #visualizes

Ages range between 18 to 60 years old, most people seem to be in their late 30s or early 40s, with some mid 20s mixed in.

In [ ]:
# correlation

corr = data.corr()
highly_corr_features = corr.index[abs(corr["Purchased"])>0.0]
plt.figure(figsize=(10,10))
map = sns.heatmap(data[highly_corr_features].corr(),annot=True)

In [ ]:
predictors = data.drop(['Purchased'],axis=1) # Features
target = data['Purchased'] # Target variable

In [ ]:
from sklearn.model_selection import train_test_split
predictors_teach, predictors_test, target_teach, target_test = sklearn.model_selection.train_test_split(predictors, target, test_size=0.3, random_state=1) # 70% training and 30% test

from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(criterion="gini",min_impurity_decrease=0.02,max_depth=4 ,min_samples_leaf=20)

decision_tree = decision_tree.fit(predictors_teach, target_teach)

In [ ]:
dot_data = StringIO()

export_graphviz(decision_tree, out_file=dot_data,
                filled=True, rounded=True,impurity=False, proportion=True,precision=2,
                special_characters=True, feature_names=predictors.columns, class_names=['3','4','5','6','7','8']) #["really bad","bad","ok","good","really good"])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

In [ ]:
#confusion matrix
prediction = decision_tree.predict(predictors_test)

from sklearn import metrics

accuracy = metrics.accuracy_score(target_test, prediction)

print("accuracy_score:",accuracy)

In [ ]:
len(prediction)

confusion_matrix = metrics.confusion_matrix(target_test, prediction)

print(confusion_matrix)